## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kodiak,US,57.7900,-152.4072,33.31,86,100,19.57,light snow
1,1,Esperance,AU,-33.8667,121.9000,62.44,70,100,13.94,overcast clouds
2,2,Husavik,IS,66.0449,-17.3389,27.50,76,100,10.49,overcast clouds
3,3,Dikson,RU,73.5069,80.5464,-12.96,98,13,8.50,few clouds
4,4,Norman Wells,CA,65.2820,-126.8329,-3.96,77,75,0.00,light snow


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Esperance,AU,-33.8667,121.9000,62.44,70,100,13.94,overcast clouds
5,5,Codrington,AU,-38.2667,141.9667,65.35,52,100,7.09,overcast clouds
7,7,Busselton,AU,-33.6500,115.3333,60.40,63,0,16.15,clear sky
11,11,Benguela,AO,-12.5763,13.4055,79.77,78,95,5.01,moderate rain
12,12,Rikitea,PF,-23.1203,-134.9692,78.69,69,5,13.15,clear sky
15,15,Faya,SA,18.3851,42.4509,73.51,38,0,5.55,clear sky
17,17,Cape Town,ZA,-33.9258,18.4232,82.81,9,0,9.22,clear sky
19,19,Lebu,CL,-37.6167,-73.6500,60.30,72,92,4.05,overcast clouds
20,20,Metro,ID,-5.1131,105.3067,75.99,91,18,3.85,moderate rain
23,23,Ribeira Grande,PT,38.5167,-28.7000,66.60,63,20,18.41,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                361
City                   361
Country                357
Lat                    361
Lng                    361
Max Temp               361
Humidity               361
Cloudiness             361
Wind Speed             361
Current Description    361
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis=0,how="any")
clean_df.count()

City_ID                357
City                   357
Country                357
Lat                    357
Lng                    357
Max Temp               357
Humidity               357
Cloudiness             357
Wind Speed             357
Current Description    357
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Esperance,AU,62.44,overcast clouds,-33.8667,121.9000,
5,Codrington,AU,65.35,overcast clouds,-38.2667,141.9667,
7,Busselton,AU,60.40,clear sky,-33.6500,115.3333,
11,Benguela,AO,79.77,moderate rain,-12.5763,13.4055,
12,Rikitea,PF,78.69,clear sky,-23.1203,-134.9692,
15,Faya,SA,73.51,clear sky,18.3851,42.4509,
17,Cape Town,ZA,82.81,clear sky,-33.9258,18.4232,
19,Lebu,CL,60.30,overcast clouds,-37.6167,-73.6500,
20,Metro,ID,75.99,moderate rain,-5.1131,105.3067,
23,Ribeira Grande,PT,66.60,few clouds,38.5167,-28.7000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Esperance,AU,62.44,overcast clouds,-33.8667,121.9000,Hospitality Esperance
5,Codrington,AU,65.35,overcast clouds,-38.2667,141.9667,Codrington Gardens Bed And Breakfast
7,Busselton,AU,60.40,clear sky,-33.6500,115.3333,Observatory Guest House
11,Benguela,AO,79.77,moderate rain,-12.5763,13.4055,Hotel Praia Morena
12,Rikitea,PF,78.69,clear sky,-23.1203,-134.9692,People ThankYou
...,...,...,...,...,...,...,...
692,Misratah,LY,71.96,overcast clouds,32.3754,15.0925,Almasa Hotel
693,Djambala,CG,83.79,broken clouds,-2.5447,14.7533,Hotel Ravin D'or
695,Luau,AO,77.45,light rain,-10.7073,22.2247,Casa- Bernardo Cândido
699,Matara,LK,79.72,few clouds,5.9485,80.5353,Amaloh Boutique Resort


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
<dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))